# Assignment 1

Authors: 
- Andreas Rosenquist (s214604)
- Felix Lund Frandsen ()
- Kasper Rønberg ()


## Assignment 1.1: Exploring WS and BA models

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import random
import numpy as np

### Part 1:

#### Exercises: Did you really read the text? Answer the following questions (no calculations needed) in your IPython notebook.

- What's the problem with random networks as a model for real-world networks according to the argument in section 3.5 (near the end)?

In random networks no clear outliers are apparent.

- List the four regimes that characterize random networks as a function of ⟨k⟩

Subcritical where k < 1, Critical Point where k = 1, Supercritcal Regime where k > 1, and Connected Regime where k > lnN

- According to the book, why is it a problem for random networks (in terms of being a model for real-world networks) that the degree-dependent clustering C(k)
 decreases as a function of k
 in real-world networks?

 Due to random models not capturing the clustering coefficient, they become poor models for real networks.

#### Exercises: WS edition.

#### First, let's use networkx to play around with WS graphs. Use nx.watts_strogatz_graph to generate 3 graphs with 500 nodes each, average degree = 4, and rewiring probablity  p=0,0.1, and  1.Calculate the average shortest path length  ⟨d⟩ for each one.

In [ ]:
N = 500
k = 4
p = [0, 0.1, 1]

WG0 = nx.watts_strogatz_graph(N, k, p[0])
WG1 = nx.watts_strogatz_graph(N, k, p[1])
WG2 = nx.watts_strogatz_graph(N, k, p[2])  

In [ ]:
WG0_avg_path_length = nx.average_shortest_path_length(WG0)
WG1_avg_path_length = nx.average_shortest_path_length(WG1)
WG2_avg_path_length = nx.average_shortest_path_length(WG2)
print("Average path length for p=0: ", WG0_avg_path_length)
print("Average path length for p=0.1: ", WG1_avg_path_length)
print("Average path length for p=1: ", WG2_avg_path_length)

#### Describe what happens to the network when  p=1.

Increasing the paramter p will increase the randomness of the network, i.e. reducing the clustering of the random network. A intermediate p will resemble the small world phenomenon of random networks, as the wiring of the edges will be more random and not only focus on the k-nearest neighbours. At p=1 we see that the average shortest path length increase significantly from p=0.1, this would indicate a futher reduction in the clustering. In other words, at p=1 for the Watt-Strogatz random network it will fully neglect the characteristc of wiring to close neighbours, but instead  choosing uniformly at random to wire an edge to any node in the network. This will naturally result in shorter paths as the network becomes more intertwined.

In [ ]:
extracted_ds = {}

ps = [0, 0.01, 0.03, 0.05, 0.1, 0.2]
N = 500
k = 4

for i in range(50):
    for p in ps:
        WSG = nx.watts_strogatz_graph(N, k, p)
        avg_path_length = nx.average_shortest_path_length(WSG)
        if p not in extracted_ds:
            extracted_ds[p] = [avg_path_length] 
        else:
            extracted_ds[p].append(avg_path_length)


In [ ]:
# code block brought to you by chatGPT
extracted_ds_avg = {p: sum(lengths)/len(lengths) for p, lengths in extracted_ds.items()}
extracted_ds_sd = {p: (sum((x - extracted_ds_avg[p]) ** 2 for x in lengths) / len(lengths))**0.5 for p, lengths in extracted_ds.items()}

In [ ]:
ds_avg = list(extracted_ds_avg.values())
ds_sd = list(extracted_ds_sd.values())

In [ ]:
plt.errorbar(ps, ds_avg, yerr=ds_sd, fmt='o', ecolor='r', capthick=2, color='black')
plt.plot(ps, ds_avg, linestyle='-', color='black')
plt.xlabel('Rewiring Probability (p)')
plt.ylabel('Average Shortest Path Length ($<d>$)')
plt.title('Average Shortest Path Length vs Rewiring Probability')
figure_text = """
Figure 1: Average Shortest Path Length, <d>, as a function of Rewiring Probability, p, in a Watts-Strogatz Graph. The figure highlights that an increase in rewiring probability p,
reduces clustering of the random network which in turn affects the average path lengths - reducing the average path length between nodes.
"""
plt.figtext(0.5, -0.15, 
            figure_text,
            wrap=True, 
            horizontalalignment='center', 
            fontsize=10
            )
plt.show()

### Part 2:

In [ ]:
# Initialize the BA graph with two connected nodes
BA = nx.Graph()
BA.add_node(1)
BA.add_node(2)
BA.add_edge(1, 2)

# Loop from 3 to 5000+1 to populate the graph with 5000 nodes
for i in range(3, 5000+1):
    # Add the new node
    BA.add_node(i)
    # We choose an existing node to connect to, with probability proportional to its degree (here we use itertools.chain to flatten the edge list)
    chosen_node = random.choice(list(itertools.chain.from_iterable(edge for edge in BA.edges)))
    # Add the edge between the new node and the chosen existing node
    BA.add_edge(i, chosen_node)


In [ ]:
# Plot BA graph
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(BA, k=0.2, iterations=100, seed=420)
nx.draw(BA, pos=pos, node_size=7, width=0.5)
plt.title(label="Home-Brewed Barabasi-Albert Random Graph")
plt.show()

In [ ]:
# Gather edges for each node
edges_count = {}
for i in range(1, 5001):
    edges_count[i] = len(BA.edges(i))

In [ ]:
# code block brought to you by chatGPT
# Finds max and min degree nodes and their degrees
max_degree_node = max(edges_count, key=edges_count.get)
print("Node with highest degree:", max_degree_node)
print("Degree of that node:", edges_count[max_degree_node])

min_degree_node = min(edges_count, key=edges_count.get)
print("Node with lowest degree:", min_degree_node)
print("Degree of that node:", edges_count[min_degree_node])

In [ ]:
# Create bins and edges based on degree counts and bin ranges based on maximum degree
bins, edges = np.histogram(
                        list(edges_count.values()), 
                        bins=range(1,  edges_count[max_degree_node]+2)
                        )

In [ ]:
plt.figure(figsize=(10, 4))
plt.scatter(edges[:-1], bins, marker='v')
plt.ylabel('Frequency')
plt.xlabel('Degree (k)')
plt.title('Degree Distribution of Home-Brewed Barabasi-Albert Random Network')
plt.figtext(0.5, -0.10, 
            "Figure 2: Degree Distribution of a Home-Brewed Barabasi-Albert Random Network (linear scale). The distribution shows the frequency of nodes having a specific degree k.",
            wrap=True, 
            horizontalalignment='center', 
            fontsize=10
            )
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.scatter(edges[:-1], bins, marker='v')
plt.ylabel('Frequency')
plt.xlabel('Degree (k)')
plt.title('Degree Distribution of Home-Brewed Barabasi-Albert Random Network')
plt.yscale('log')
plt.xscale('log')
plt.figtext(0.5, -0.10, 
            "Figure 2: Degree Distribution of a Home-Brewed Barabasi-Albert Random Network (log scale). The distribution shows the frequency of nodes having a specific degree k.",
            wrap=True, 
            horizontalalignment='center', 
            fontsize=10
            )
plt.show()